## Plastic Waste Pyrolysis Case Study 🚮♻️

This notebook explores the capabilities of PULPO in the context of (simplified) plastic waste pyrolysis. This case study focuses on the conversion of plastic waste into valuable products through pyrolysis technology.

Updated by Fabian Lechtenberg, 30.03.2024

The key functionalities demonstrated include:

- Technology Choice 🛠️
- Foreground vs. Background 🎭🌐
- Regional Choice 🌍
- Alternative Indicators 🔄
- Technosphere Capacity Constraint ⛓️
- Biosphere Capacity Constraint 🌱⛔
- Indicator Constraints 📏
- Supply vs. Demand 🔄💼

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">🚨 Disclaimer: Implementation of Inventories</h2>
</div>

In typical LCA applications, especially for attributional / comparative studies, inventories are pre-established or pre-build by the practitioner within LCA databases. The manual steps shown here to create or adjust inventories for the plastic waste pyrolysis case study are illustrative, aimed at demonstrating process customization for this specific context.

In [ ]:
import brightway2 as bw

**Setup Brightway2 Project**

Initializes the Brightway2 project named "PULPO Final". It checks if the Ecoinvent database "cutoff38" is available.


In [ ]:
# Setup Brightway2 project
bw.projects.set_current("pulpo")
if "cutoff38" not in bw.databases:
    print("Ecoinvent database not found. Please import it before running this script.")
else:
    db = bw.Database("cutoff38")
    biosphere = bw.Database("biosphere3")

**System Inventory Overview**

The diagram illustrates the inventory flows for the polypropylene (PP) pyrolysis process. Inputs such as electricity, deionized water, and chromium steel are quantified on the left, feeding into the central PP pyrolysis activity. The "PP waste collected" is considered a burden-free input, serving as a placeholder for later implementing technical constraints. Outputs of the process include pyrolysis oil and emissions of CO2 and H2O. 🔄🚮🛢️➡️🌎

<div style="text-align: center;">
    <img src="pictures/PP_pyrolysis_inventory.png" alt="Image Alt Text" style="width: 50%;" />
</div>

**Create or Check for Waste Polypropylene Process**

Retrieve or create a new activity for the collection of PP waste.

In [ ]:
# Check if the empty PP waste process already exists
try:
    pp_waste_process = db.get("pp_waste")
    print("PP waste process already exists, skipping the addition.")
except:
    pp_waste_process = db.new_activity(
        code="pp_waste",
        name="PP Waste collected",
        unit="kilogram",
        location="Europe without Switzerland",
    )
    pp_waste_process["reference product"] = "PP waste"
    pp_waste_process.save()

**Create or Check for PP Pyrolysis Process**

Retrieve or create a new activity for the PP pyrolysis process, including setting up various technosphere and biosphere exchanges.

In [ ]:
# Check if the PP pyrolysis process already exists
try:
    pp_pyrolysis_process = db.get("pp_pyrolysis")
    print("PP pyrolysis process already exists, skipping the addition.")
except:
    pp_pyrolysis_process = db.new_activity(
        code="pp_pyrolysis",
        name="PP pyrolysis process",
        unit="kilogram",
        location="Europe without Switzerland",
    )
    pp_pyrolysis_process["reference product"] = "pyrolysis oil"

    # Add technosphere exchanges
    # PP waste
    pp = db.search("PP Waste collected")[0]
    pp_pyrolysis_process.new_exchange(input=pp.key, amount=(26.6)/23.9, type='technosphere').save()
    # Add electricity input
    electricities = [act for act in db if "market group for electricity, high voltage" in act['name'] and act['location'] == "Europe without Switzerland"]
    electricity = electricities[0] if electricities else None
    if electricity:
        pp_pyrolysis_process.new_exchange(input=electricity.key, amount=(7.881+7.368e-2)/23.9, type='technosphere').save()
    # Add water input
    water = db.search("market for water, deionised")[0]
    pp_pyrolysis_process.new_exchange(input=water.key, amount=(1e-4)/23.9, type='technosphere').save()  
    # Add equipment input
    steel = db.search("market for steel, chromium steel 18/8")[0]
    pp_pyrolysis_process.new_exchange(input=steel.key, amount=(1.075e-4)/23.9, type='technosphere').save()
    # Add biosphere exchanges
    co2 = biosphere.search("carbon dioxide")[0]
    pp_pyrolysis_process.new_exchange(input=co2.key, amount=(5.278)/23.9, type='biosphere').save()
    h2o = biosphere.search("water, in air")[0]
    pp_pyrolysis_process.new_exchange(input=h2o.key, amount=(3.129)/23.9, type='biosphere').save()

    # Save the activity
    pp_pyrolysis_process.save()
    print("PP pyrolysis process and its exchanges have been successfully added.")

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Demonstration 🚀</h2>
</div>

With the inventories now defined, we're stepping into the practical demonstration of our plastic waste pyrolysis case study.


In these code snippets, we're setting up the environment and preparing for an analysis with the Pulpo framework:

In [ ]:
import os
import sys
sys.path.append('../') ### not necessary if pulpo is pip installed!
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

This segment of the code is focused on defining key parameters and paths for the analysis:

- **Project and Database**: Specifies the Brightway2 project (`"pulpo"`) and database (`"cutoff38"`) to be used for the life cycle assessment.
- **Methods**: Sets the impact assessment method to IPCC 2013 GWP 100a, a widely recognized standard for evaluating global warming potential.
- **Directory Paths**: Configures the working directory (`directory`) relative to the notebook's location for storing and accessing data files.
- **GAMS Path**: Establishes the file path to the GAMS executable (`GAMS_PATH`), integrating the GAMS optimization software with our Python environment for performing complex optimization tasks.

<span style="color:red">**Note:**</span> Instead of GAMS, HiGHS, an open-source linear and nonlinear solver, can also be used for optimization tasks.

In [ ]:
project = "pulpo"
database = "cutoff38"
methods = "('IPCC 2013', 'climate change', 'GWP 100a')"

# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = "C:/GAMS/37/gams.exe"

Initialize a Pulpo object by passing the key parameters and paths to the PulpoOptimizer class:

In [ ]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Technology choice 🛠️ and fore- vs. background optimization 🎭🌐</h2>
</div>

In the forthcoming segment, we delve into a comparative demonstration focused on alternative technologies:

- **Pyrolysis Oil** 🚮⛽
- **Fossil Diesel** 🦕⛽
- **Biodiesel** 🌱⛽

We'll also examine two contrasting optimization strategies:

1. **Decoupled (Offline) Optimization** Assessing the market in a vacuum, disconnected from wider technosphere / economy dynamics.
2. **Integrated (Simultaneous) Optimization** Conducting a holistic technosphere optimization, considering the ripple effects and feedback from market changes.

**Retrieving Necessary Activities for Assessment**

Below, we fetch the essential activities for our analysis. The system under review is illustrated in the figure that follows. 📉👇

<div style="text-align: center;">
    <img src="pictures/PP_pyrolysis_system_boundaries.png" alt="Image Alt Text" style="width: 50%;" />
</div>

**Diesel Market**


In [ ]:
activities = ["market for diesel"]
reference_products = ["diesel"]
locations = ["Europe without Switzerland"]

In [ ]:
diesel_market = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)[0]

In [ ]:
from helper import copy_activity
diesel_market_foreground = copy_activity(diesel_market)

**Diesel Production**

In [ ]:
activities = ["diesel production, petroleum refinery operation"]
reference_products = ["diesel"]
locations = ["Europe without Switzerland"]

In [ ]:
diesel_fossil = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)[0]

In [ ]:
diesel_fossil_foreground = copy_activity(diesel_fossil)

In [ ]:
activities = ["soybean oil refinery operation"]
reference_products = ["soybean oil, refined"]
locations = ["RoW"]

**Biodiesel Production**

In [ ]:
diesel_bio = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)[0]

In [ ]:
diesel_bio_foreground = copy_activity(diesel_bio)

In [ ]:
activities = ["PP pyrolysis process"]
reference_products = ["pyrolysis oil"]
locations = ["Europe without Switzerland"]

**Pyrolysis Oil Production**

In [ ]:
diesel_pp = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)[0]

Once all inventories have been fetched, load the LCI data to the Pulpo worker:

In [ ]:
pulpo_worker.get_lci_data()

**No optimization**

In this segment, we're setting up a comparative LCA for three different types of diesel without optimization.

For each fuel type, we are evaluating the environmental impact associated with the production of 1 kilogram. The assessment is conducted using a suite of impact assessment methods:

- **Global Warming Potential (GWP)**: Assessed over 100 years as per IPCC 2013.
- **Land Use Change GWP100**: Evaluates climate impact specifically from land use and land use change.
- **Freshwater Eutrophication**: Measures the potential impact of nutrient enrichment in freshwater ecosystems.
- **Ionising Radiation Human Health**: Estimates the exposure efficiency to ionizing radiation and its effect on human health.

In [ ]:
functional_units = [
    {diesel_bio: 1},   # 1 kg of biodiesel
    {diesel_pp: 1},     # 1 kg of diesel from PP
    {diesel_fossil: 1} # 1 kg of fossil diesel
]

methods_list = [
    ("IPCC 2013", "climate change", "GWP 100a"),
    ("EF v3.0", "climate change: land use and land use change", "global warming potential (GWP100)"),
    ("EF v3.0", "eutrophication: freshwater", "fraction of nutrients reaching freshwater end compartment (P)"),
    ("EF v3.0", "ionising radiation: human health", "human exposure efficiency relative to u235"),
]

my_calculation_setup = {'inv': functional_units, 'ia': methods_list}
bw.calculation_setups['diesel_comparison'] = my_calculation_setup
mlca = bw.MultiLCA('diesel_comparison')

In [ ]:
import pandas as pd
from IPython.display import display

# Define the names of the activities and methods based on your setup
activity_names = ['Biodiesel', 'Diesel from PP', 'Fossil Diesel']

# Convert the mlca.results array to a DataFrame
results_df = pd.DataFrame(
    mlca.results, 
    index=activity_names, 
    columns=methods_list
)

# Define a custom function to apply colors based on ranking within each column
def highlight_by_column_rank(s):
    is_max = s == s.max()
    is_min = s == s.min()
    # Assign colors: red for max, green for min, orange for others
    return ['background-color: red' if v else 'background-color: green' if m else 'background-color: orange' for v, m in zip(is_max, is_min)]

# Apply the styling for each column
styled_df = results_df.style.format("{:.2e}")
styled_df = styled_df.apply(highlight_by_column_rank, axis=0)

# Display the styled DataFrame
display(styled_df)

The table showcases the comparative LCA results across different diesel options, color-coded to indicate the lowest (green), middle (orange), and highest (red) environmental impacts in each category. Biodiesel shows a higher GWP 100a, while Diesel from PP demonstrates potential as the least impactful in the GWP category. As we move to optimize for GWP, the preliminary data suggests that pyrolysis diesel might minimize the impact. However, the extent of this benefit, especially when considering technical constraints and the distinction between decoupled and integrated optimization results, remains to be explored in the following sections.

**Foreground assessment** 🎭

Specify the demand of the assessment as 1 kg of diesel from the *foreground* diesel market:

In [ ]:
demand = {diesel_market_foreground: 1}

Specify the choices of the assessment as the three diesel production alternatives:

In [ ]:
choices  = {'diesel': {diesel_bio_foreground: 10,
                       diesel_fossil_foreground: 10,
                       diesel_pp: 10}}

Instantiate and solve the problem:

In [ ]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

Summarize the results:

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

**Background Assessment** 🌐

Specify the demand of the assessment as 1 kg of diesel from the *original* diesel market which is connected to all other processes of the LCI database:

In [ ]:
demand = {diesel_market: 1}

In [ ]:
choices  = {'diesel': {diesel_bio: 10,
                       diesel_fossil: 10,
                       diesel_pp: 10}}

In [ ]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The optimization analysis reaffirms our expectations: pyrolysis oil emerges as the preferable choice, with its corresponding alpha (α) parameter set to "1", indicating full selection in the optimized mix.

When contrasting the "foreground" (market-specific) with the "background" (economy-wide) optimization scenarios, we observe a marginal disparity: the foreground scenario yields a GWP of **0.373366 kg CO<sub>2</sub>e**, while the background scenario presents a marginally lower GWP of **0.373292 kg CO<sub>2</sub>e**. This small difference is attributable to the ripple effects of the diesel market's lower impacts cascading upstream, thereby slightly reducing the overall environmental burden.

Notably, the scaling vector for PP pyrolysis, s<sub>diesel_PP</sub>, which represents the level of utilization of pyrolysis oil, remains consistent at "1" in the foreground scenario—signifying exclusive use within the market. In contrast, the background scenario, which integrates the technology economy-wide, reflects a subtle uptick in utilization to s<sub>diesel_PP</sub> = 1.000713. This represents a 0.07% increment, hinting at a small but noteworthy expansion in demand when the technology is adopted across the broader economy. 

The following illustration shows two example paths that are affected by the change in the market composition, which are omitted in the foreground scenario 📉👇 

<div style="text-align: center;">
    <img src="pictures/PP_ripple_effect.png" alt="Image Alt Text" style="width: 50%;" />
</div>

The more direct route originates from the diesel market and proceeds to the freight train transport activity. This activity subsequently feeds into the transport market, which in turn serves as an input back to the diesel market, thus establishing a feedback loop. Such a loop allows any changes in the environmental impact of the diesel market to feed back to the diesel market, influencing its own impact.

A secondary feedback loop is illustrated as follows: it starts from the "market for transport, freight train" and moves to the market for lignite. This, in sequence, contributes to the production of electricity from lignite, which is part of the German electricity mix. The German mix then plays a role in the European electricity mix, which, in turn, is a key input for the plastic pyrolysis process. This interconnected loop exemplifies how changes within one segment can propagate through the entire energy supply chain, ultimately affecting the pyrolysis process and the diesel market itself.

See [this showcase](electricity_showcase.ipynb) for an example with a much larger ripple effect! 

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Regional choice 🌍</h2>
</div>

Our previous assessment concentrated on evaluating **technology choices**. Now, we shift our focus to **regional selections**, simultaneously considering various European sourcing options for electricity. By specifying the PP pyrolysis process to accept electricity from the broader EUR electricity market group, we allow for the possibility of drawing power from any eligible national electricity markets within Europe.

From a mathematical standpoint, the distinction between technological and regional choices is subtle, as both essentially involve selecting from explicitly modeled **processes** within the Life Cycle Inventory (LCI) database. Whether differentiating by technology or by geographic origin, the underlying computational approach remains consistent.

In our expanded analysis, we introduce "electricity" as a new choice alongside "diesel." Now, the electricity used in the PP pyrolysis process can come from any national market within Europe, not just the European mix. This addition opens up 40 different alternatives, complicating the decision-making process as each choice has implications for multiple impact categories and can directly or indirectly affect the other technologies. Determining the optimal choice in this intricate network of possibilities presents a more complex challenge  📉👇 

<div style="text-align: center;">
    <img src="pictures/PP_regional_choice.png" alt="Image Alt Text" style="width: 50%;" />
</div>

In [ ]:
country_codes = [
    "FR", "DE", "GB", "IT", "ES", "PL", "UA", "SE", "NO", "NL",
    "FI", "BE", "AT", "CZ", "GR", "RO", "PT", "HU", "RS", "BG",
    "BY", "DK", "IE", "SK", "IS", "HR", "SI", "BA", "LT", "MK",
    "AL", "EE", "LU", "LV", "MD", "ME", "XK", "MT", "GI"
]

Now let's use a Pulpo function to retrieve the corresponding electricity markets. The function "retrieve_activities" of the worker allows us to specify the activity names, the reference product, or the locations and will return the best match:

In [ ]:
activities = ["market for electricity, high voltage"]
reference_products = ["electricity, high voltage"]
# Get regional markets
electricity_markets = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=country_codes)
# Get european market
electricity_markets.append(pulpo_worker.retrieve_activities(activities=["market group for electricity, high voltage"],
                                                            reference_products=reference_products,
                                                            locations=["Europe without Switzerland"])[0])

print(f"A total of {len(electricity_markets)} electricity markets have been retrieved.")

Now that the electricity markets have been retrieved, let's specify the new choice "electricity":

In [ ]:
choices['electricity']  = {market: 10 for market in electricity_markets}

With the expanded choces, proceed to re-instatiate the problem and re-solve:

In [ ]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

For the regional choice "Albania" (AL) has been chosen, supplying a total of **0.4565 kWh**, of which 0.3328 kWh go to the PP pyrolysis and the rest is used somewhere else in the technosphere. As a result, the optimized system can further reduce the GWP from **0.3733 kg CO2** (only technology choice) to **0.2385 kg CO2** (simultaneous technology and regional choice).  

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Alternative Indicators 🔄</h2>
</div>

Judging from the "*no optimization*" case from before, the best choice for the "**Freshwater Eutrophication (EF v3.0)**" category will be **fossil diesel**. In this segment, let's assess what happens if we optimize for this category, while simultaneously allow for regional choices to be active.

To that end, create a new worker with all the methods:

In [ ]:
methods_list = {
    "('IPCC 2013', 'climate change', 'GWP 100a')": 0,
    "('EF v3.0', 'climate change: land use and land use change', 'global warming potential (GWP100)')": 0,
    "('EF v3.0', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)')": 1,
    "('EF v3.0', 'ionising radiation: human health', 'human exposure efficiency relative to u235')": 0,
}

The provided dictionary is utilized within the optimization framework to allocate specific weights to various impact categories. Each key in this dictionary represents a distinct environmental impact category, as identified by its methodological definition (e.g., "IPCC 2013" for climate change or "EF v3.0" for specific Environmental Footprint categories). The value assigned to each key represents the weight that the optimization algorithm should give to that particular impact category in the decision-making process.

- The weight of 0 for most categories (climate change, land use change's GWP, and ionising radiation's human health impact) indicates that these categories do not influence the optimization's outcome. However, they are evaluated (calculated) alongside the optimization and their final value can be printed and assessed.
- The weight of 1 assigned to "eutrophication: freshwater" signifies that the optimization process exclusively focuses on minimizing the impact in this category, while still calculating the impacts for the other categories alongside.

In [ ]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods_list, directory)
pulpo_worker.get_lci_data()

Let's re-instantiate and re-solve the problem (same choices, same demand):

In [ ]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

#### Notable Findings:
Contrary to initial expectations based solely on technology choice, it has been observed that not fossil diesel but pyrolysis oil, powered with electricity from **Iceland (IS)**, significantly reduces the eutrophication impact from **2.86e-05 kg PO4e** to **2.00e-05  kg PO4e**.

However, it's important to note that these findings do not definitively establish Iceland as the optimal location for diesel production of this nature. Instead, they prompt a deeper examination of Iceland's local conditions that contribute to this environmental advantage. Notably, Iceland's energy production is predominantly sourced from hydro and geothermal energy, with a particularly high reliance on geothermal sources. This unique energy landscape suggests that the extensive use of geothermal energy plays a critical role in positioning Iceland as a potentially favorable region for minimizing eutrophication impacts.

Accordingly, these results open avenues for further analysis and understanding, potentially revealing broader insights into environmental optimization and regional energy strategies.

In [ ]:
# Reset the objective function to IPCC GWP
methods_list = {
    "('IPCC 2013', 'climate change', 'GWP 100a')": 1,
    "('EF v3.0', 'climate change: land use and land use change', 'global warming potential (GWP100)')": 0,
    "('EF v3.0', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)')": 0,
    "('EF v3.0', 'ionising radiation: human health', 'human exposure efficiency relative to u235')": 0,
}

pulpo_worker = pulpo.PulpoOptimizer(project, database, methods_list, directory)
pulpo_worker.get_lci_data()

<div style="text-align: center; background-color: #FFA500; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #FFFFFF;">Additional Constraints</h2>
</div>

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Technosphere Capacity Constraints ⛓️</h2>
</div>

In 2020, the demand for diesel products stood at **292.6 million tons** ([source](https://www.fuelseurope.eu/uploads/files/modules/documents/file/1663766146_TWxIVaKHUN4dlyHl7hNvrUpf0Jwzbbe6nI9DMDBa.pdf)), necessitating **1.113 kg of PP (polypropylene)** for the production of 1 kg of pyrolysis oil. Within the European Union, the total post-consumer plastic waste collected, both mixed and separately, amounted to **29.5 million tons** ([source](https://plasticseurope.org/knowledge-hub/plastics-the-facts-2022/)). This presents a clear mismatch between demand and available supply.

Significantly, over **50% of plastic waste** is amassed through mixed waste collection schemes, with the separately collected fraction rarely categorized by plastic type. This makes acquiring "pure" PP for pyrolysis a challenging endeavor. 

However, for the purposes of our assessment, we'll operate under the optimistic assumption that the separately collected fraction of plastics includes **16.6% PP**, and that it's feasible to sort and extract this PP entirely for pyrolysis usage. Under this premise, approximately **2.4 million tons of raw PP material** could be rendered available for pyrolysis.

We will proceed by incorporating this raw material limitation as a **technosphere constraint** in our analysis:

In [ ]:
# Adjust the final demand for an economy-wide perspective
demand = {diesel_market: 292.6e9}

In [ ]:
# Re-define the choices and set a reasonably high capacity constraint
# Note that we are not considering regional choices here!
choices  = {'diesel': {diesel_bio: 1e15,
                       diesel_fossil: 1e15,
                       diesel_pp: 1e15}} 

To incorporate the constraint on plastic availability, we must initially retrieve the relevant activity. Following this, we'll construct an "upper_limit" dictionary that assigns the previously derived value:

In [ ]:
pp_waste = pulpo_worker.retrieve_activities(activities=['PP Waste collected'])[0]

In [ ]:
upper_limit = {pp_waste: 2.4e9}

With this, we re-instatiate and re-solve the problem:

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_limit)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit)

As expected, under ther technosphere constraint we obtain that 2.15 million ton of pyrolysis oil are produced and 291.10 million ton of fossil diesel. The overall minimized GWP stands at 137.7595 Mt CO2e. 

Now let's see what happens if we add further constraints to the problem. 

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Indicator / Impact Constraints 📏</h2>
</div>

For instance, a regulation aiming to reduce the ionising radiation impact by 5%! Using the previous result as baseline (6.5521e+10 kBq U235e) the target should be 6.2245e+10 kBq U235e. This constraint can be added as follows:

In [ ]:
upper_imp_limit = {"('EF v3.0', 'ionising radiation: human health', 'human exposure efficiency relative to u235')": 6.2245e+10}

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_limit, upper_imp_limit=upper_imp_limit)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit, zeroes=True)

Since the process minimizing ionising radiation has already reached its limit through the technosphere constraint, the solver selects the next best process to comply with the additional impact constraint. This results in the use of **39.09 million ton of imported biodiesel**.

The trade-off to comply with the constraint is an increased GWP from **137.7595 Mt CO2e** to **428.6267 Mt CO2e**! In other words, to reduce one impact category by 5%, another one increased by 211.13%. 

This kind of assessment can help to identify technological and systemic bottlenecks in the multi-objective design of production systems.

In [ ]:
pulpo_worker.save_results(choices=choices, demand=demand, name='plastic_showcase_constrained.xlsx')

 <div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Biosphere capacity constraints 🌱</h2>
</div>

From the results saved in the spreadsheet "plastic_showcase_constrained.xlsx" we can identify (as expected) that the biodiesel production has a very large impact on land transformation. The biosphere entry "Transformation, from annual crop" takes a value of 109,643 km² which is slightly larger than the entire country of Iceland.

Pulpo enable the restriction of biosphere flows as well. Let's repeat the previous assessment, targeting this time ionising radiation as objective, and figure out which is the possible reduction if we allow only 10,000 km² of land transformation: 

In [ ]:
pulpo_worker.method["('IPCC 2013', 'climate change', 'GWP 100a')"] = 0
pulpo_worker.method["('EF v3.0', 'ionising radiation: human health', 'human exposure efficiency relative to u235')"] = 1

In [ ]:
transformation = pulpo_worker.retrieve_envflows(activities=["Transformation, from annual crop"])[0]

In [ ]:
upper_elem_limit = {transformation: 1e10}

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_limit, upper_elem_limit=upper_elem_limit)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

It can be observed that, as before, the maximum amount of pyrolysis oil is selected. The biodiesel however now is produced with **17.83 million ton**, reducing the overall ionising radiation to 6.522251e+10 kBq U235e (0.46% reduction).

---

**Disclaimer**: This example is designed for demonstrative purposes only, to showcase the functionalities of Pulpo. The figures, scenarios, and scales presented are hypothetical and may not accurately reflect real-world conditions. Any resemblance to actual data or outcomes is purely coincidental and should not be considered as accurate representations of environmental impacts or optimization results.

---


 <div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Supply vs. demand 🔄💼</h2>
</div>

Pulpo introduces a feature that pivots from defining a final demand to setting a final supply. This approach is particularly useful in scenarios where the end demand remains uncertain, yet there's clarity on a process's production capacity. Under such circumstances, the aim is to maximize production, anticipating that the eventual demand will exceed this capacity. For example, we might know the production limit for pyrolysis oil but only have an estimated demand for market-sourced diesel.

To demonstrate this capability, we'll explore specifying a final **supply** of 1 kg of market diesel, contrasting this with a scenario based on a 1 kg diesel **demand**. It's reasonable to anticipate a somewhat reduced total impact in the supply specification scenario, considering that it accounts for diesel's internal consumption, which is normally added to the final demand.

In [ ]:
pulpo_worker.method["('IPCC 2013', 'climate change', 'GWP 100a')"] = 1
pulpo_worker.method["('EF v3.0', 'ionising radiation: human health', 'human exposure efficiency relative to u235')"] = 0

In [ ]:
supply = {diesel_market: 1}

The final supply is specified in the instatiation by setting a lower and upper limit of the scaling vector to the same value:

In [ ]:
pulpo_worker.instantiate(choices=choices, lower_limit=supply, upper_limit=supply)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

As expected, the total impact is **slightly** lower with 0.373<span style="color: red;">026</span> kg CO<sub>2</sub> compared to 0.373<span style="color: red;">292</span> kg CO<sub>2</sub>e.
